#  Colab에서 PySpark 실행하는 방법
- 참조 : https://medium.com/@TheITspace/running-pyspark-on-google-colab-2552435972b3

In [1]:
# Colab에서 PySpark 사용하는 방법 1
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 12.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=320cddfa70bb5bbcf2b89617c7883c29c6413a1cf478d3a0a7b77e8e921944f2
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
# # Colab에서 PySpark 사용하는 방법 2
# !apt-get install openjdk-8-jdk-headless -qq 
# !wget -q !wget -q https://downloads.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz  
# !tar -xf spark-3.3.2-bin-hadoop3.tgz
# !pip install -q findspark

# import os
# import findspark

# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

# findspark.init()
# findspark.find()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("demo")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [4]:
spark

## ngrok를 사용하여 Spark UI 접속

In [5]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

--2023-03-22 12:56:27--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8162036 (7.8M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   7.78M  35.9MB/s    in 0.2s    

2023-03-22 12:56:28 (35.9 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [8162036/8162036]



In [6]:
!tar xf ngrok-v3-stable-linux-amd64.tgz

In [16]:
#  !./ngrok authtoken <authtoken > # https://dashboard.ngrok.com/
!./ngrok authtoken 68EgMVst4xkntqLofArnK_7VuGPawLqhayeXRFfNLtH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [22]:
get_ipython().system_raw('./ngrok http 4050 &')

In [23]:
!ps -ef | grep ngrok

root        2621       1  2 13:01 ?        00:00:00 ./ngrok http 4050
root        2654     758  0 13:01 ?        00:00:00 /bin/bash -c ps -ef | grep ngrok
root        2656    2654  0 13:01 ?        00:00:00 grep ngrok


In [26]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://044d-35-196-217-58.ngrok.io


## 데이터 로딩
- 데이터 출처 : https://www.kaggle.com/sdolezel/black-friday

In [27]:
df = spark.read.csv("train.csv", header=True, inferSchema=True)

In [28]:
df.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



In [29]:
df.show(5)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|
|1000001| P00087842|     F|0-17|        10|            A|                         2|             0|                12|              null|              null|    1422|
|100

In [30]:
df.count()

22589

## PySpark 사용법

### 특정 컬럼 데이터 보기

In [31]:
df.select("User_ID","Gender","Age","Occupation").show(5)

+-------+------+----+----------+
|User_ID|Gender| Age|Occupation|
+-------+------+----+----------+
|1000001|     F|0-17|        10|
|1000001|     F|0-17|        10|
|1000001|     F|0-17|        10|
|1000001|     F|0-17|        10|
|1000002|     M| 55+|        16|
+-------+------+----+----------+
only showing top 5 rows



### 컬럼

In [32]:
df.describe().show()

+-------+------------------+----------+------+-----+------------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|summary|           User_ID|Product_ID|Gender|  Age|        Occupation|City_Category|Stay_In_Current_City_Years|     Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|         Purchase|
+-------+------------------+----------+------+-----+------------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|  count|             22589|     22589| 22589|22589|             22589|        22589|                     22589|              22589|             22589|             15375|              6786|            22589|
|   mean|1001753.7059188101|      null|  null| null| 8.271636637301341|         null|        1.4850456501206841|0.40134578777280977| 5.303112134224623| 9.86543089430894

### 범주형 컬럼(Categorical columns)의 유일값 확인

In [33]:
df.select("City_Category").distinct().show()

+-------------+
|City_Category|
+-------------+
|            B|
|            C|
|            A|
+-------------+



### Groupby 집계


In [34]:
from pyspark.sql import functions as F
df.groupBy("City_Category").agg(F.sum("Purchase")).show()

+-------------+-------------+
|City_Category|sum(Purchase)|
+-------------+-------------+
|            B|     85896015|
|            C|     66335351|
|            A|     56538339|
+-------------+-------------+



### Null value 확인 및 처리

In [35]:
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|      0|         0|     0|  0|         0|            0|                         0|             0|                 0|              7214|             15803|       0|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+



In [36]:
df = df.fillna({'Product_Category_2':0, 'Product_Category_3':0})

In [37]:
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|      0|         0|     0|  0|         0|            0|                         0|             0|                 0|                 0|                 0|       0|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+



## 데이터 저장

In [38]:
df.write.csv("pre_processed_data")

In [39]:
df.rdd.getNumPartitions()

1

## Pandas 데이터프레임으로 저장

In [40]:
df_pd = df.toPandas()
df_pd.to_csv("pandas_pre_processed_data.csv")